In [ ]:
import requests
import json

In [ ]:
ID_KEY="_id"
CODE_KEY="code"
PAYLOAD_KEY="item"
QUERY_KEY="search"

class InvocationException(Exception):
    def __init__(self, response):
        self.response = response
        super().__init__(f"got response code {response.status_code} from remote side")

class CatalogClient: 
    def __init__(self, catalog_url):
        self.url = catalog_url
    
    def invoke(self, request_type, payload_dict, path_variable):
        this_url = self.url+'/sda/'+request_type
        resp = None
        if payload_dict is None:
            if path_variable is None:
                resp = requests.get(this_url)
            elif request_type == 'retrieve':
                resp = requests.get(this_url+'/{}'.format(path_variable))
            else:
                resp = requests.delete(this_url+'/{}'.format(path_variable))
#             resp = requests.post(this_url)
        else:
            if request_type == 'update':
               
                resp = requests.put(this_url+'/{}'.format(path_variable), json=payload_dict)
            else:
                resp = requests.post(this_url, json=payload_dict)
        if resp.status_code != 200:
            raise InvocationException(resp)
        return resp
    

    def create_request(self, data_uri, scope, data_type, metadata, return_id=True):
        this_dict = dict()
        this_dict["data_uri"] = data_uri
        this_dict["scope"] = scope
        if data_type is not None: 
            this_dict["data_type"] = data_type
        if metadata is not None: 
            this_dict["metadata"] = metadata
        req = self.invoke('create', this_dict, None)
        if return_id: 
            ans_dict = json.loads(req.text)
            return ans_dict.get(ID_KEY, None)
        else:
            return req.text
    
    def delete_request(self, identifier):
        this_dict = {ID_KEY:identifier}
        try:
#             req = self.invoke('delete', None, this_dict)
            req = self.invoke('delete', None, identifier)
            return True
        except InvocationException as e:
            if e.response.status_code == 500:
                return False
            else:
                raise e
            
    
    def retrieve_request(self, identifier):
#         this_dict = {ID_KEY:identifier}
        try:
#             req = self.invoke('retrieve', this_dict)
            req = self.invoke('retrieve', None, identifier)
            ans_dict = req.text
            return req.text
        except InvocationException as e:
            if e.response.status_code == 500:
                return None
            else:
                raise e
    
    def update_request(self, identifier, data_uri=None, scope=None, data_type=None, metadata=None, return_id=True):
        if data_uri is None and scope is None and data_type is None and metadata is None:
            return 'Nothing to update'
        this_dict = dict()
        if data_uri is not None: 
            this_dict["data_uri"] = data_uri
        if scope is not None: 
            this_dict["scope"] = scope
        if data_type is not None: 
            this_dict["data_type"] = data_type
        if metadata is not None: 
            this_dict["metadata"] = metadata
#         this_dict[ID_KEY] = identifier
        try:
#             req = self.invoke('update', this_dict)
            req = self.invoke('update', this_dict, identifier)
            if return_id: 
                ans_dict = json.loads(req.text)
                return ans_dict.get(ID_KEY, None)
            else:
                return req.text
#             return req.text
        except InvocationException as e:
            if e.response.status_code == 500:
                return None
            else:
                raise e
        
    def list_all(self):
        resp = self.invoke('list', None, None)
        return resp.text
    
    def purge(self):
        resp = self.invoke('purge', None, None)
          

In [ ]:
def my_parse(this_string):
    this_list = list()
    brack_counter=0
    start_index=0
    for index,char in enumerate(this_string[1:-1]):
        if char=='{':
            brack_counter=brack_counter+1
        elif char=='}':
            brack_counter=brack_counter-1
            if brack_counter==0:
                item=this_string[start_index+1:index+2]
                this_list.append(item)
                start_index=index+1
    return this_list

def find_match(obj_list, scope, data_type, name):
    for obj in obj_list:
        if obj["scope"] == scope and obj["data_type"]==data_type:
            mdata = obj["metadata"]
            if mdata is not None and mdata["name"] == name:
                return obj
        

In [ ]:
def get_file_contents(filename):
    f = open(filename, 'rb').read()
    hex_encoded = binascii.hexlify(f).decode('utf-8')
    return hex_encoded

def save_to_file(filename, bin_string):
    with open(filename, 'wb') as f:
        f.write(bin_string)

        
def get_mnist_dataset(train=True):
    dataset = datasets.MNIST(root=data_dir,
                               train=train,
                               download=True,
                               transform=transforms.ToTensor())
    return dataset

